# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read csv
cities = "output_data/cities.csv"
city_df = pd.read_csv(cities)
city_df.head(1)

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%),Date
0,Rikitea,PF,-23.12,-134.97,69.08,66,5.84,0,2020-08-28


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = city_df[["Latitude", "Longitude"]]

# Store humidity
humidity = city_df["Humidity (%)"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the DataFrame to find your ideal weather condition. 
# A max temperature lower than 80 degrees but higher than 70.
# Humidity between 30-60%
# Wind speed less than 10 mph.
# Zero cloudiness.
good_weather_df = city_df.loc[(city_df["Temperature (F)"]<=80) & 
                             (city_df["Temperature (F)"]>=70) &
                             (city_df["Wind Speed (mph)"]<=10) &
                             (city_df["Cloudiness (%)"]==0) &
                             (city_df["Humidity (%)"]<=60) &
                             (city_df["Humidity (%)"]>30)].dropna()
good_weather_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%),Date
1,Sarakhs,IR,36.54,61.16,71.60,33,4.70,0,2020-08-28
422,Kavos,GR,38.83,22.83,71.01,51,1.99,0,2020-08-28
475,Sabha,JO,32.33,36.50,75.20,60,6.80,0,2020-08-28


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# create dataframe for hotel list
hotel_df = good_weather_df.loc[:,["City","Country","Latitude","Longitude"]]
hotel_df["Hotel Name"] = ""
hotel_df.head(1)

,City,Country,Latitude,Longitude,Hotel Name
1,Sarakhs,IR,36.54,61.16,


In [6]:
# find the first hotel for each city located within 5000 meters of the coordinates
# set parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "lodging",
    "keywork":"hotel",
    "language":"en",
    "radius":5000,
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{latitude},{longitude}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"First hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
#     # Wait 1 sec to make another api request to avoid SSL Error
#     time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 1: Sarakhs.
First hotel in Sarakhs is Sarakhs Abrisham Hotel.
------------
Retrieving Results for Index 422: Kavos.
First hotel in Kavos is Kavos Villa.
------------
Retrieving Results for Index 475: Sabha.
First hotel in Sabha is مجمع الدبيس التجاري.
------------
-------End of Search-------


In [7]:
# display hotel_df with hotel names
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
1,Sarakhs,IR,36.54,61.16,Sarakhs Abrisham Hotel
422,Kavos,GR,38.83,22.83,Kavos Villa
475,Sabha,JO,32.33,36.50,مجمع الدبيس التجاري


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))